In [20]:
# pip install charmap

In [21]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, DMatrix

import json
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import classification_report

In [22]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Load your dataset
with open('dataset/haveMD_302.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Add a new column to classify if 'Generic policy' is present or not
df['has_generic_policy'] = df['SecurityPolicy_content_category'].apply(
    lambda x: 'Generic policy' in x if isinstance(x, list) else False
)

# Separate data into features (X) and target (y)
X = df.drop('has_generic_policy', axis=1)
y = df['has_generic_policy']

# Stratified train-test split 70-30
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=y, random_state=42)

# Split temp_df into validation and test sets 20-10
validation_df, test_df = train_test_split(temp_df, test_size=(1/3), stratify=temp_df['has_generic_policy'], random_state=42)

# Function to save DataFrame to JSON without escaping characters and with UTF-8 encoding
def save_json_without_escape(filepath, dataframe):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(dataframe.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

# Save datasets to JSON files
save_json_without_escape('train_dataset.json', train_df)
save_json_without_escape('validation_dataset.json', validation_df)
save_json_without_escape('test_dataset.json', test_df)

# Print the counts and distributions for verification
print('Training set distribution and count:')
print(train_df['has_generic_policy'].value_counts(normalize=True))
print(f"Total projects in Training set: {len(train_df)}")

print('Validation set distribution and count:')
print(validation_df['has_generic_policy'].value_counts(normalize=True))
print(f"Total projects in Validation set: {len(validation_df)}")

print('Test set distribution and count:')
print(test_df['has_generic_policy'].value_counts(normalize=True))
print(f"Total projects in Test set: {len(test_df)}")


Training set distribution and count:
has_generic_policy
True     0.687204
False    0.312796
Name: proportion, dtype: float64
Total projects in Training set: 211
Validation set distribution and count:
has_generic_policy
True     0.683333
False    0.316667
Name: proportion, dtype: float64
Total projects in Validation set: 60
Test set distribution and count:
has_generic_policy
True     0.677419
False    0.322581
Name: proportion, dtype: float64
Total projects in Test set: 31
